In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Project-MONAI/MONAI.git

Cloning into 'MONAI'...
remote: Enumerating objects: 36913, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 36913 (delta 68), reused 109 (delta 45), pack-reused 36754
Receiving objects: 100% (36913/36913), 64.09 MiB | 12.44 MiB/s, done.
Resolving deltas: 100% (29628/29628), done.


In [ ]:
%cd MONAI/


/content/MONAI


In [ ]:
!pip install -e '.[all]'

Obtaining file:///content/MONAI
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 M

In [ ]:
model_dir = '/content/drive/MyDrive/template/our-git-project/Cross_Val_Results_final_3'

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict

from monai.apps import download_url
from monai.utils import set_determinism
from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.config import print_config
from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    ToTensord,
)

from monai.metrics import DiceMetric
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)

print_config()

MONAI version: 1.3.0+42.gd5585c34
Numpy version: 1.23.5
Pytorch version: 2.1.0+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: d5585c34fc44f46af9ffb28522ce44c7ed869654
MONAI __file__: /content/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 4.0.2
scikit-image version: 0.19.3
scipy version: 1.11.4
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: 4.6.6
TorchVision version: 0.16.0+cu118
tqdm version: 4.66.1
lmdb version: 1.4.1
psutil version: 5.9.5
pandas version: 1.5.3
einops version: 0.7.0
transformers version: 4.21.3
mlflow version: 2.9.0
pynrrd version: 1.0.0
clearml version: 1.13.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:
set_determinism(seed=960)

In [ ]:
use_pretrained = True


if use_pretrained:

    dst = '/content/drive/MyDrive/template/our-git-project/Pretrained_Results_final_v2/best_model.pt'

    pretrained_path = os.path.normpath(dst)


In [ ]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    EnsureChannelFirstd,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    NormalizeIntensityd,
    CropForegroundd,
    RandShiftIntensityd,
    RandFlipd,
    FgBgToIndicesd,
    RandRotate90d,
    RandCropByPosNegLabeld,
    Activations,



)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

# Training Hyper-params
lr = 1e-5
max_iterations = 10000
eval_num = 100

original_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'
mask_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'

# List of image file paths
images  = sorted(glob(os.path.join(original_data_folder, '*_orig.nii.gz')))
segs  = sorted(glob(os.path.join(mask_data_folder, '*_masks.nii.gz')))

data_dicts  = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(images, segs)]
train_files, test_files = data_dicts[:-25], data_dicts[-25:]


train_transforms = Compose(

[
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #NormalizeIntensityd(keys="vol", nonzero=True, channel_wise=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            RandCropByPosNegLabeld(
            keys=["vol", "seg"],
            label_key="seg",
            spatial_size=(64, 64, 64),
            pos=4,
            neg=1,
            num_samples=5,
            image_key="vol",
            image_threshold=0,
            ),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[0],
            prob=0.1,
        ),
        RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[1],
            prob=0.1,
        ),
        RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[2],
            prob=0.1,
        ),
        RandRotate90d(
            keys=["vol", "seg"],
            prob=0.3,
            max_k=3,
            spatial_axes=[0, 2],
        ),
        ToTensord(keys=["vol", "seg"]),

        ]
)
val_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #NormalizeIntensityd(keys="vol", nonzero=True, channel_wise=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            RandCropByPosNegLabeld(
            keys=["vol", "seg"],
            label_key="seg",
            spatial_size=(64, 64, 64),
            pos=4,
            neg=1,
            num_samples=5,
            image_key="vol",
            image_threshold=0,
            ),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[0],
            prob=0.1,
        ),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[1],
            prob=0.1,
        ),

        RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[2],
            prob=0.1,
        ),
        RandRotate90d(
            keys=["vol", "seg"],
            prob=0.1,
            max_k=3,
            spatial_axes=[0, 2],
        ),
        ToTensord(keys=["vol", "seg"]),


        ]
)



monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [ ]:
from sklearn.model_selection import KFold
import os
from monai.networks.nets import UNet,UNETR
from monai.networks.layers import Act, Norm
from monai.losses import DiceLoss, DiceCELoss
import torch

# Number of folds
num_folds = 5

# Use sklearn's KFold to split the data into 5 folds
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_datasets = list(kf.split(train_files))

# Lists to store results for each fold
fold_results = []

In [ ]:
# Iterate over folds
for fold, (train_indices, val_indices) in enumerate(fold_datasets):
    print(f"\nFold {fold + 1}/{num_folds}")

    if(fold > 1):
        del train_ds, train_loader
        torch.cuda.empty_cache()

        del val_ds, val_loader
        torch.cuda.empty_cache()

    # Create training and validation datasets for the current fold
    train_files = [data_dicts[i] for i in train_indices]
    val_files = [data_dicts[i] for i in val_indices]

    train_ds = CacheDataset(
    data=train_files,
    transform=train_transforms,
    cache_num=24,
    cache_rate=1.0,
    num_workers=4,
    )

    val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_num=6, cache_rate=1.0, num_workers=4)

    # Create data loaders for training and validation sets
    train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNETR(
            in_channels=1,
            out_channels=2,
            img_size=(64, 64, 64),
            feature_size=16,
            hidden_size=768,
            mlp_dim=3072,
            num_heads=12,
            pos_embed="conv",
            norm_name="instance",
            res_block=True,
            dropout_rate=0.0,
        ).to(device)

    # Load ViT backbone weights into UNETR
    if use_pretrained is True:
            print("Loading Weights from the Path {}".format(pretrained_path))
            vit_dict = torch.load(pretrained_path)
            vit_weights = vit_dict["state_dict"]

            # Remove items of vit_weights if they are not in the ViT backbone (this is used in UNETR).
            # For example, some variables names like conv3d_transpose.weight, conv3d_transpose.bias,
            # conv3d_transpose_1.weight and conv3d_transpose_1.bias are used to match dimensions
            # while pretraining with ViTAutoEnc and are not a part of ViT backbone.
            model_dict = model.vit.state_dict()

            vit_weights = {k: v for k, v in vit_weights.items() if k in model_dict}
            model_dict.update(vit_weights)
            model.vit.load_state_dict(model_dict)
            del model_dict, vit_weights, vit_dict
            print("Pretrained Weights Succesfully Loaded !")

    elif use_pretrained is False:
            print("No weights were loaded, all weights being used are randomly initialized!")

    model.to(device)

    loss_function = DiceCELoss(to_onehot_y=True, softmax=True,  jaccard=True)
    torch.backends.cudnn.benchmark = True
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)

    post_label = AsDiscrete(to_onehot=2)
    post_pred = AsDiscrete(argmax=True, to_onehot=2)
    dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    global_step = 0
    dice_val_best = 0.0
    global_step_best = 0
    epoch_loss_values = []
    metric_values = []

    def validation(epoch_iterator_val):
        model.eval()
        dice_vals = []

        with torch.no_grad():
            for _step, batch in enumerate(epoch_iterator_val):
                val_inputs, val_labels = (batch["vol"].cuda(), batch["seg"].cuda())
                val_outputs = sliding_window_inference(val_inputs, (64, 64, 64), 4, model)
                val_labels_list = decollate_batch(val_labels)
                val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
                val_outputs_list = decollate_batch(val_outputs)
                val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
                dice_metric(y_pred=val_output_convert, y=val_labels_convert)
                dice = dice_metric.aggregate().item()
                dice_vals.append(dice)
                epoch_iterator_val.set_description("Validate (%d / %d Steps) (dice=%2.5f)" % (global_step, 10.0, dice))

            dice_metric.reset()

        mean_dice_val = np.mean(dice_vals)

        return mean_dice_val


    def train(global_step, train_loader, dice_val_best, global_step_best):
        model.train()
        epoch_loss = 0
        step = 0
        epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
        for step, batch in enumerate(epoch_iterator):
            step += 1
            x, y = (batch["vol"].cuda(), batch["seg"].cuda())
            logit_map = model(x)
            loss = loss_function(logit_map, y)
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()
            epoch_iterator.set_description("Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss))

            if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
                epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)
                dice_val = validation(epoch_iterator_val)

                epoch_loss /= step
                epoch_loss_values.append(epoch_loss)
                metric_values.append(dice_val)
                if dice_val > dice_val_best:
                    dice_val_best = dice_val
                    global_step_best = global_step
                    torch.save(model.state_dict(), os.path.join(model_dir, "best_metric_model.pth"))
                    print(
                        "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val)
                    )
                else:
                    print(
                        "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(
                            dice_val_best, dice_val
                        )
                    )

                plt.figure(1, (12, 6))
                plt.subplot(1, 2, 1)
                plt.title("Iteration Average Loss")
                x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
                y = epoch_loss_values
                plt.xlabel("Iteration")
                plt.plot(x, y)
                plt.grid()
                plt.subplot(1, 2, 2)
                plt.title("Val Mean Dice")
                x = [eval_num * (i + 1) for i in range(len(metric_values))]
                y = metric_values
                plt.xlabel("Iteration")
                plt.plot(x, y)
                plt.grid()
                plt.savefig(os.path.join(model_dir, "unet_finetune_quick_update.png"))
                plt.clf()
                plt.close(1)

            global_step += 1

        return global_step, dice_val_best, global_step_best


    while global_step < max_iterations:
        global_step, dice_val_best, global_step_best = train(global_step, train_loader, dice_val_best, global_step_best)
    model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_model.pth")))

    # Print metrics for the current fold
    print(f"Fold {fold + 1}/{num_folds} completed, best_metric: {dice_val_best:.4f} "
          f"at iteration: {global_step_best}")
    #fold_results = fold_results.append(dice_val_best)

# Print overall cross-validation results
print(f"\nOverall cross-validation completed, best_metric: {dice_val_best:.4f} "
      f"at iteration: {global_step_best}")




Fold 1/5


Loading dataset: 100%|██████████| 6/6 [00:04<00:00,  1.39it/s]
monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.


Loading Weights from the Path /content/drive/MyDrive/template/our-git-project/Pretrained_Results_final_v2/best_model.pt
Pretrained Weights Succesfully Loaded !


Validate (68 / 10 Steps) (dice=0.65176): 100%|██████████| 17/17 [00:07<00:00,  2.31it/s]


Model Was Saved ! Current Best Avg. Dice: 0.6615856991094702 Current Avg. Dice: 0.6615856991094702


...

Model Was Not Saved ! Current Best Avg. Dice: 0.8945056725950802 Current Avg. Dice: 0.8798549035016228


Validate (6800 / 10 Steps) (dice=0.85903): 100%|██████████| 17/17 [00:08<00:00,  2.09it/s]


Model Was Saved ! Current Best Avg. Dice: 0.8974884713397306 Current Avg. Dice: 0.8974884713397306


Training (6879 / 10000 Steps) (loss=0.65215):  18%|█▊        | 12/68 [00:07<00:34,  1.64it/s]


RuntimeError: ignored

#5000 Steps
Fold1 - 0.9086 Fold2 - 0.87 Fold3 - 0.895 Fold4 - 0.888 Fold5 - 87.19
#10000 Steps
Fold1 - 0.9379 Fold2 - 0.9170 Fold3-

In [ ]:
plt.figure(1, (12, 6))
plt.subplot(1, 2, 1)
plt.title("Iteration Average Loss")
x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.grid()
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [eval_num * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.grid()
plt.show()

In [ ]:
from glob import glob
original_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'
mask_data_folder = '/content/drive/MyDrive/template/our-git-project/preprocessed_images/'

# List of image file paths
vol  = sorted(glob(os.path.join(original_data_folder, '*_orig.nii.gz')))
segs  = sorted(glob(os.path.join(mask_data_folder, '*_masks.nii.gz')))

pixdim=(1.0, 1.0, 1.0)
a_min=-200
a_max=200
spatial_size=[150,150,150]

data_dicts = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(vol, segs)]
train_files, test_files = data_dicts[:-9], data_dicts[-9:]


test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #NormalizeIntensityd(keys="vol", nonzero=True, channel_wise=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            RandCropByPosNegLabeld(
            keys=["vol", "seg"],
            label_key="seg",
            spatial_size=(64, 64, 64),
            pos=4,
            neg=1,
            num_samples=5,
            image_key="vol",
            image_threshold=0,),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[0],
            prob=0.5,
        ),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[0],
            prob=0.5,
        ),
            RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[1],
            prob=0.5,
        ),

        RandFlipd(
            keys=["vol", "seg"],
            spatial_axis=[2],
            prob=0.5,
        ),
        RandRotate90d(
            keys=["vol", "seg"],
            prob=0.5,
            max_k=3,
        ),
        ToTensord(keys=["vol", "seg"]),


        ]
    )

test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
test_loader = DataLoader(test_ds, batch_size=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
    in_channels=1,
    out_channels=2,
    img_size=(64, 64, 64),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="conv",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)


In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

In [ ]:
from monai.utils import first
sw_batch_size = 4
roi_size = (64, 64, 64)
with torch.no_grad():
    test_patient = first(test_loader)
    t_volume = test_patient['vol'].cuda()
    val_labels = test_patient['seg'].cuda()

    val_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
    val_labels_list = decollate_batch(val_labels)
    val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
    val_outputs_list = decollate_batch(val_outputs)
    val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
    dice_metric(y_pred=val_output_convert, y=val_labels_convert)
    dice = dice_metric.aggregate().item()
    print("Test Dice Value: ",dice)

In [ ]:
sw_batch_size = 20
roi_size = (64, 64, 64)
with torch.no_grad():
    test_patient = first(test_loader)
    t_volume = test_patient['vol']
    #t_segmentation = test_patient['seg']

    test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
    sigmoid_activation = Activations(sigmoid=True)
    test_outputs = sigmoid_activation(test_outputs)
    test_outputs = test_outputs > 0.69

    for i in range(31):
        # plot the slice [:, :, 80]
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(test_patient["vol"][0, 0, :, :, i], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"label {i}")
        plt.imshow(test_patient["seg"][0, 0, :, :, i] != 0)
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(test_outputs.detach().cpu()[0, 1, :, :, i])
        plt.show()

In [ ]:
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    KeepLargestConnectedComponent,
    LabelToContour,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
    RemoveSmallObjects,

)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob


print_config()

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
        test_patient = first(test_loader)
        t_volume = test_patient['vol']
        #t_segmentation = test_patient['seg']

        test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
        sigmoid_activation = Activations(sigmoid=True)
        test_outputs = sigmoid_activation(test_outputs)
        test_outputs_1 = test_outputs > 0.69

        for i in range(51):
            # plot the slice [:, :, 80]
            plt.figure("check", (20, 4))
            plt.subplot(1, 7, 1)
            plt.title(f"image {i}")
            plt.imshow(test_patient['vol'].detach().cpu()[0, 0, :, :, i], cmap="gray")
            plt.subplot(1, 7, 2)
            plt.title(f"argmax {i}")
            argmax = [AsDiscrete(argmax=True)(j) for j in decollate_batch(test_outputs)]
            plt.imshow(argmax[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 3)
            plt.title(f"largest {i}")
            largest = [KeepLargestConnectedComponent(applied_labels=[1])(j) for j in argmax]
            plt.imshow(largest[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 4)
            plt.title(f"contour {i}")
            contour = [LabelToContour()(j) for j in largest]
            plt.imshow(contour[0].detach().cpu()[0, :, :, i])
            plt.subplot(1, 7, 5)
            plt.title(f"map image {i}")
            map_image = contour[0] + t_volume.to(device)[0]
            plt.imshow(map_image.detach().cpu()[0, :, :, i], cmap="gray")
            plt.subplot(1, 7, 6)
            plt.title(f"output {i}")
            plt.imshow(test_outputs_1.detach().cpu()[0, 1, :, :, i])
            plt.subplot(1, 7, 7)
            plt.title(f"label {i}")
            plt.imshow(test_patient["seg"][0, 0, :, :, i] != 0)
            plt.show()

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
    for i, val_data in enumerate(test_loader):
        roi_size = (64, 64, 64)
        sw_batch_size = 4
        val_data = val_data["vol"].to(device)
        val_output = sliding_window_inference(val_data, roi_size, sw_batch_size, model)
        # plot the slice [:, :, 80]
        plt.figure("check", (20, 4))
        plt.subplot(1, 5, 1)
        plt.title(f"image {i}")
        plt.imshow(val_data.detach().cpu()[0, 0, :, :, 30], cmap="gray")
        plt.subplot(1, 5, 2)
        plt.title(f"argmax {i}")

        argmax = [AsDiscrete(argmax=True)(i) for i in decollate_batch(val_output)]
        plt.imshow(argmax[0].detach().cpu()[0, :, :, 30])
        plt.subplot(1, 5, 3)
        plt.title(f"largest {i}")
        largest = [KeepLargestConnectedComponent(applied_labels=[1])(i) for i in argmax]
        plt.imshow(largest[0].detach().cpu()[0, :, :, 30])
        plt.subplot(1, 5, 4)
        plt.title(f"contour {i}")
        contour = [LabelToContour()(i) for i in largest]
        plt.imshow(contour[0].detach().cpu()[0, :, :, 30])
        plt.subplot(1, 5, 5)
        plt.title(f"map image {i}")
        map_image = contour[0] + val_data[0]
        plt.imshow(map_image.detach().cpu()[0, :, :, 30], cmap="gray")

        plt.show()